In [2]:
import re
import os
import glob
import zipfile
import requests
from urllib.request import urlretrieve
import json
import pandas as pd

In [10]:
# Necessary metadata
article_id = 14096681  # this is the unique identifier of the article on figshare
url = f"https://api.figshare.com/v2/articles/{article_id}"
headers = {"Content-Type": "application/json"}
output_directory = "figshareairline"

In [4]:
response = requests.request("GET", url, headers=headers)
data = json.loads(response.text)  # this contains all the articles data, feel free to check it out
files = data["files"]             # this is just the data about the files, which is what we want
files

[{'id': 26579150,
  'name': 'daily_rainfall_2014.png',
  'size': 58863,
  'is_link_only': False,
  'download_url': 'https://ndownloader.figshare.com/files/26579150',
  'supplied_md5': 'fd32a2ffde300a31f8d63b1825d47e5e',
  'computed_md5': 'fd32a2ffde300a31f8d63b1825d47e5e'},
 {'id': 26579171,
  'name': 'environment.yml',
  'size': 192,
  'is_link_only': False,
  'download_url': 'https://ndownloader.figshare.com/files/26579171',
  'supplied_md5': '060b2020017eed93a1ee7dd8c65b2f34',
  'computed_md5': '060b2020017eed93a1ee7dd8c65b2f34'},
 {'id': 26586554,
  'name': 'README.md',
  'size': 5422,
  'is_link_only': False,
  'download_url': 'https://ndownloader.figshare.com/files/26586554',
  'supplied_md5': '61858c6cc0e6a6d6663a7e4c75bbd88c',
  'computed_md5': '61858c6cc0e6a6d6663a7e4c75bbd88c'},
 {'id': 26766812,
  'name': 'data.zip',
  'size': 814041183,
  'is_link_only': False,
  'download_url': 'https://ndownloader.figshare.com/files/26766812',
  'supplied_md5': 'b517383f76e77bd03755a63a8f

In [5]:
%%time
files_to_dl = ["data.zip"]  # feel free to add other files here
for file in files:
    if file["name"] in files_to_dl:
        os.makedirs(output_directory, exist_ok=True)
        urlretrieve(file["download_url"], output_directory + file["name"])

CPU times: user 5.49 s, sys: 3.71 s, total: 9.2 s
Wall time: 1min 47s


In [13]:
%%time
with zipfile.ZipFile(os.path.join(output_directory, "data.zip"), 'r') as f:
    f.extractall(output_directory + '/data')

CPU times: user 14.6 s, sys: 1.14 s, total: 15.8 s
Wall time: 16.6 s


In [14]:
%ls -ltr figshareairline/data

total 10540336
-rw-r--r--   1 bananabook  staff   95376895 Mar 28 14:53 MPI-ESM-1-2-HAM_daily_rainfall_NSW.csv
-rw-r--r--   1 bananabook  staff   94960113 Mar 28 14:53 AWI-ESM-1-1-LR_daily_rainfall_NSW.csv
-rw-r--r--   1 bananabook  staff   82474546 Mar 28 14:53 NorESM2-LM_daily_rainfall_NSW.csv
-rw-r--r--   1 bananabook  staff  127613760 Mar 28 14:53 ACCESS-CM2_daily_rainfall_NSW.csv
-rw-r--r--   1 bananabook  staff  232118894 Mar 28 14:53 FGOALS-f3-L_daily_rainfall_NSW.csv
-rw-r--r--   1 bananabook  staff  330360682 Mar 28 14:53 CMCC-CM2-HR4_daily_rainfall_NSW.csv
-rw-r--r--   1 bananabook  staff  254009247 Mar 28 14:53 MRI-ESM2-0_daily_rainfall_NSW.csv
-rw-r--r--   1 bananabook  staff  235661418 Mar 28 14:53 GFDL-CM4_daily_rainfall_NSW.csv
-rw-r--r--   1 bananabook  staff  294260911 Mar 28 14:53 BCC-CSM2-MR_daily_rainfall_NSW.csv
-rw-r--r--   1 bananabook  staff  295768615 Mar 28 14:53 EC-Earth3-Veg-LR_daily_rainfall_NSW.csv
-rw-r--r--   1 bananabook  staff  328852379 Mar 28 14:53 C

In [16]:
%%time
### just listing to get an idea how individual file looks like
df = pd.read_csv(
    "figshareairline/data/ACCESS-CM2_daily_rainfall_NSW.csv", dtype={"TailNum": "str"}
)
df

CPU times: user 1.02 s, sys: 126 ms, total: 1.15 s
Wall time: 1.19 s


,time,lat_min,lat_max,lon_min,lon_max,rain (mm/day)
0,1889-01-01 12:00:00,-36.25,-35.00,140.625,142.50,3.293256e-13
1,1889-01-02 12:00:00,-36.25,-35.00,140.625,142.50,0.000000e+00
2,1889-01-03 12:00:00,-36.25,-35.00,140.625,142.50,0.000000e+00
3,1889-01-04 12:00:00,-36.25,-35.00,140.625,142.50,0.000000e+00
4,1889-01-05 12:00:00,-36.25,-35.00,140.625,142.50,1.047658e-02
...,...,...,...,...,...,...
1932835,2014-12-27 12:00:00,-30.00,-28.75,151.875,153.75,2.951144e-02
1932836,2014-12-28 12:00:00,-30.00,-28.75,151.875,153.75,2.257118e-01
1932837,2014-12-29 12:00:00,-30.00,-28.75,151.875,153.75,1.204670e-01
1932838,2014-12-30 12:00:00,-30.00,-28.75,151.875,153.75,2.632404e-02


In [21]:
%%time
## here we are using a normal python way for merging the data 
import pandas as pd

files = glob.glob('figshareairline/data/*.csv')
df = pd.concat((pd.read_csv(file, index_col=0)
                .assign(model=re.findall(r'[^\/]+[^_]*', file)[0])
                for file in files)
              )
df.to_csv("figshareairline/combined_data.csv")

CPU times: user 6min 10s, sys: 10.8 s, total: 6min 21s
Wall time: 6min 29s


In [20]:
df = pd.read_csv("figshareairline/combined_data.csv")
df.head()

,time,lat_min,lat_max,lon_min,lon_max,rain (mm/day),model
0,1889-01-01 12:00:00,-35.439867,-33.574619,141.5625,143.4375,4.244226e-13,figshareairline/data/MPI-ESM-1-2-HAM
1,1889-01-02 12:00:00,-35.439867,-33.574619,141.5625,143.4375,4.217326e-13,figshareairline/data/MPI-ESM-1-2-HAM
2,1889-01-03 12:00:00,-35.439867,-33.574619,141.5625,143.4375,4.498125e-13,figshareairline/data/MPI-ESM-1-2-HAM
3,1889-01-04 12:00:00,-35.439867,-33.574619,141.5625,143.4375,4.251282e-13,figshareairline/data/MPI-ESM-1-2-HAM
4,1889-01-05 12:00:00,-35.439867,-33.574619,141.5625,143.4375,4.270161e-13,figshareairline/data/MPI-ESM-1-2-HAM
